In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer, NGram
import re


mypath = "cookbook_text/*.txt"

def flatten(pair):
    text = re.sub(r'\n\s*\n','\n',pair[1],re.MULTILINE)
    text = re.sub(' +', ' ', text)
    text = re.sub("\n", "", text)
    text = re.sub("\r", "", text)
    return [[[name for name in pair[0].split('/')][-1] ,text]]

# sc = SparkContext()
spark = SparkSession \
        .builder \
        .appName("LSH") \
        .getOrCreate()
sc = spark.sparkContext
data = sc.wholeTextFiles(mypath).flatMap(flatten)
# print(data)
cook_book_df = data.toDF(['title','content'])

#Tokenize the data 
tokenizer = Tokenizer(inputCol="content", outputCol="words")
tokenized = tokenizer.transform(cook_book_df)
newdf = tokenized.select("title","content", "words")

#Fit a CountVectorizerModel
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=100000, minDF=2)
model = cv.fit(newdf)
newdf = model.transform(newdf)

# Fit a MinHashLSH model 
mh = MinHashLSH(inputCol="features", outputCol="hashValues", seed=12345).setNumHashTables(3)
model = mh.fit(newdf)
newdf = model.transform(newdf)
print("Number of Files - ")
print(newdf.count())
print("Data types of columns - ")
print(newdf.dtypes)
newdf.show()

similarity_matrix = model.approxSimilarityJoin(newdf, newdf, 3, "JaccardDistance").select(col("datasetA.title")\
                    .alias("Title A"), col("datasetB.title").alias("Title B"),\
                    col("JaccardDistance")).sort(desc("JaccardDistance")).dropDuplicates(['JaccardDistance'])
threshold = 0.9
# similarity_matrix.show()
matching_docs = similarity_matrix.filter(similarity_matrix['JaccardDistance'] < threshold)
# print(matching_docs.count())
print("Displaying matching docs having JaccardDistance < 0.9")
matching_docs.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.writeUTF(PythonRDD.scala:394)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:214)
	at org.apache.spark.api.python.PythonRDD$$anonfun$wri